In [1]:
from dask_gateway import Gateway
from dask_gateway.auth import BasicAuth
gateway = Gateway("http://147.251.21.115:30080",
    auth=BasicAuth(password="x"),
    )

In [2]:
cluster = gateway.new_cluster()

In [3]:
cluster

In [4]:
options = gateway.cluster_options()
options

In [5]:
options

In [6]:
from distributed import Client
client=Client(cluster)

/Users/todaka/miniconda3/envs/jupyter/lib/python3.10/site-packages/distributed/client.py:1274: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| lz4     | 4.0.0  | None      | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [9]:
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: http://147.251.21.115:30080/clusters/dask-gateway.ad050e1a62884390a2d347bf86d19921/status,


In [10]:
import xarray as xr

In [22]:
ds=xr.tutorial.open_dataset("air_temperature",chunks={"time":500})

In [23]:
ds=ds.persist()



In [14]:
client.get_versions(check=True)

/Users/todaka/miniconda3/envs/jupyter/lib/python3.10/site-packages/distributed/client.py:4239: UserWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| lz4     | 4.0.0  | None      | None    |
| pandas  | 1.4.2  | None      | None    |
+---------+--------+-----------+---------+
  warnings.warn(msg["warning"])


{'scheduler': {'host': {'python': '3.10.5.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '5.4.0-109-generic',
   'machine': 'x86_64',
   'processor': '',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.10.5.final.0',
   'dask': '2022.6.0',
   'distributed': '2022.6.0',
   'msgpack': '1.0.4',
   'cloudpickle': '2.1.0',
   'tornado': '6.1',
   'toolz': '0.11.2',
   'numpy': '1.22.4',
   'pandas': None,
   'lz4': None}},
 'workers': {'tls://10.244.1.5:36943': {'host': {'python': '3.10.5.final.0',
    'python-bits': 64,
    'OS': 'Linux',
    'OS-release': '5.4.0-109-generic',
    'machine': 'x86_64',
    'processor': '',
    'byteorder': 'little',
    'LC_ALL': 'None',
    'LANG': 'C.UTF-8'},
   'packages': {'python': '3.10.5.final.0',
    'dask': '2022.6.0',
    'distributed': '2022.6.0',
    'msgpack': '1.0.4',
    'cloudpickle': '2.1.0',
    'tornado': '6.1',
    'toolz': '0.11.2',
    'numpy': '1.22.4',
    'pandas'

In [15]:
from dask import delayed
import numpy as np
f=delayed(lambda x: np.exp(x*x))

#client.submit(f(2))
client.compute(f(2))



<Future: pending, key: lambda-8393f692-0d65-486b-a275-cee4e2f82e1c>

In [16]:
import dask
dask.compute(f(2))

(54.598150033144236,)

In [17]:
from distributed.diagnostics.plugin import PipInstall

In [18]:
extra_packages=["xarray","netCDF4","pooch","h5netcdf"]
plugin=PipInstall(extra_packages,restart=True)

In [19]:
client.register_worker_plugin(plugin)

{'tls://10.244.1.5:36943': {'status': 'OK'},
 'tls://10.244.3.6:43487': {'status': 'OK'}}

In [21]:
client.get_versions(packages=extra_packages,check=True)

{'scheduler': {'host': {'python': '3.10.5.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '5.4.0-109-generic',
   'machine': 'x86_64',
   'processor': '',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.10.5.final.0',
   'dask': '2022.6.0',
   'distributed': '2022.6.0',
   'msgpack': '1.0.4',
   'cloudpickle': '2.1.0',
   'tornado': '6.1',
   'toolz': '0.11.2',
   'numpy': '1.22.4',
   'pandas': None,
   'lz4': None,
   'xarray': None,
   'netCDF4': None,
   'pooch': None,
   'h5netcdf': None}},
 'workers': {'tls://10.244.1.5:39495': {'host': {'python': '3.10.5.final.0',
    'python-bits': 64,
    'OS': 'Linux',
    'OS-release': '5.4.0-109-generic',
    'machine': 'x86_64',
    'processor': '',
    'byteorder': 'little',
    'LC_ALL': 'None',
    'LANG': 'C.UTF-8'},
   'packages': {'python': '3.10.5.final.0',
    'dask': '2022.6.0',
    'distributed': '2022.6.0',
    'msgpack': '1.0.4',
    'cloudpickle': '2.1.0',
  

In [32]:
ds.air.mean(dim='time').persist()

<xarray.DataArray 'air' (lat: 25, lon: 53)>
dask.array<mean_agg-aggregate, shape=(25, 53), dtype=float32, chunksize=(25, 53), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0

argument of type 'NoneType' is not iterable
Traceback (most recent call last):
  File "/Users/todaka/miniconda3/envs/jupyter/lib/python3.10/site-packages/distributed/comm/tcp.py", line 226, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/todaka/miniconda3/envs/jupyter/lib/python3.10/site-packages/distributed/utils.py", line 761, in wrapper
    return await func(*args, **kwargs)
  File "/Users/todaka/miniconda3/envs/jupyter/lib/python3.10/site-packages/distributed/client.py", line 1392, in _handle_report
    msgs = await self.scheduler_comm.comm.read()
  File "/Users/todaka/miniconda3/envs/jupyter/lib/python3.10/site-packages/distributed/comm/tcp.py", line 242, in read
    convert_stream_closed_error(self, e)
  File "/Users/todaka/miniconda3/envs/jupyter/lib/python3.10/site-packages/distributed/comm/tcp

In [29]:
url='https://github.com/pydata/xarray-data/blob/master/air_temperature.nc?raw=true'
ds=xr.open_dataset(url,engine="netcdf4")
#ds.compute()#.mean(dim="time").compute()#.air.data.visualize()
#ds=ds.chunk({"time":500})


OSError: [Errno -75] NetCDF: Malformed or unexpected Constraint: b'https://github.com/pydata/xarray-data/blob/master/air_temperature.nc?raw=true'

In [30]:
ds.mean(dim="time").compute()#.air.data.visualize()



KilledWorker: ('original-open_dataset-0600501a96c511004c3080f12973fc5bair-2ce0cf8268eab9594c4ba765fa5feb79', <WorkerState 'tls://10.244.1.5:36915', name: dask-worker-ad050e1a62884390a2d347bf86d19921-8wphd, status: closed, memory: 0, processing: 1>)